## Practica 1 - PIAV

Integrantes: Alejandro Bolaños García y David García Díaz

### 1a. Hacer un programa que cargue una imagen y que obtenga y muestre el valor RGB de cada píxel de la imagen sobre el que esté el cursor del ratón.


In [ ]:
import cv2 as cv
import numpy as np

# Función de callback para el ratón
def mouse(event, x, y, flags, param):
    if event == cv.EVENT_MOUSEMOVE:  
        b, g, r = img[y, x]  # Se obtienen los valores de los canales BGR de la imagen en la posición (x, y)
        
        # Mostrar el valor RGB en la imagen en la esquina superior izquierda
        img_copy = img.copy()
        text = f'RGB: ({r}, {g}, {b})'
        cv.putText(img_copy, text, (10, 30), cv.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv.LINE_AA)
        cv.imshow('imagen', img_copy)

# Cargar la imagen
img = cv.imread('images/cats.png')

# Mostrar la imagen
cv.imshow('imagen', img)

# Configurar el callback del ratón
cv.setMouseCallback('imagen', mouse)

cv.waitKey(0)
cv.destroyAllWindows()

En este código, se utiliza una función de callback para el ratón para el evento "mousemove", que se activa en cada movimiento del cursor. Al detectar el evento, obtiene los valores de color en formato RGB en la posición actual del cursor y muestra estos valores en la esquina superior izquierda de la imagen. La ventana se actualiza continuamente para reflejar los valores RGB correspondientes al punto sobre el que se encuentra el cursor.

### 1b. Hacer un programa para dibujar las siguientes primitivas con el ratón:
- Líneas
- Rectángulo
- Círculos
- Especificar el color en RGB de las primitivas
- Definir el grosor de las líneas

### 2. Mejoras del programa
- Permitir crear figuras rellenas (seleccionar a través de la interfaz.
- Poder crear otras primitivas (construir polilíneas y polígonos, elipses…)
- Guardar el dibujo como un vídeo para ver cómo se realizó paso a paso
- Aportación propia: guardar imagen con el dibujo realizado y limpiar el lienzo.

In [62]:
import numpy as np
import cv2 as cv

drawing = False  # Verdadero si el ratón está presionado
mode = 0  # 0: rectángulo, 1: círculo, 2: línea, 3: polilínea, 4: polígono, 5: elipse
ix, iy = -1, -1
points = []  # Lista para almacenar puntos de polilíneas y polígonos
video_writer = None

# Lista de nombres de modos para la leyenda
mode_names = ['Rectangulo', 'Circulo', 'Linea', 'Polilinea', 'Poligono', 'Elipse']

def trackbar(x):
    pass

# Función para inicializar el video writer
def initialize_video_writer():
    global video_writer
    fourcc = cv.VideoWriter_fourcc(*'XVID')
    video_writer = cv.VideoWriter('drawing.avi', fourcc, 2.0, (950, 800))

# Función de callback para el ratón
def draw_figures(event, x, y, flags, param):
    global ix, iy, drawing, mode, points

    if event == cv.EVENT_LBUTTONDOWN:  # Se empieza a dibujar presionando el clic izquierdo
        drawing = True
        ix, iy = x, y
        if mode == 3 or mode == 4:  # Si es Polilínea o Polígono añadimos el punto a la lista
            points.append((x, y))  

    elif event == cv.EVENT_MOUSEMOVE:  
        if drawing:
            img_copy = img.copy()  # Copia de la imagen para dibujar en tiempo real
            thickness = cv.getTrackbarPos('Grosor', 'controls')
            color = (cv.getTrackbarPos('B', 'controls'), cv.getTrackbarPos('G', 'controls'), cv.getTrackbarPos('R', 'controls'))

            if mode == 0:  # Rectángulo
                cv.rectangle(img_copy, (ix, iy), (x, y), color, thickness)
            elif mode == 1:  # Círculo
                radius = int(np.sqrt((x - ix) ** 2 + (y - iy) ** 2))
                cv.circle(img_copy, (ix, iy), radius, color, thickness)
            elif mode == 2:  # Línea
                cv.line(img_copy, (ix, iy), (x, y), color, thickness)
            elif mode == 5:  # Elipse
                cv.ellipse(img_copy, (ix, iy), (abs(x - ix), abs(y - iy)), 0, 0, 360, color, thickness)
            elif mode == 3 or mode == 4:  # Polilínea o Polígono
                if len(points) > 1:
                    if thickness == -1 and mode == 4:
                        cv.fillPoly(img_copy, [np.array(points, np.int32)], color)
                    else:
                        cv.polylines(img_copy, [np.array(points, np.int32)], mode == 4, color, thickness)

            cv.putText(img_copy, f'Modo: {mode_names[mode]}', (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv.LINE_AA)
            cv.imshow('canvas', img_copy)
    
    elif event == cv.EVENT_RBUTTONDOWN:  # Completamos el polígono o la polilínea con el clic derecho
        if mode == 3 or mode == 4:
            thickness = cv.getTrackbarPos('Grosor', 'controls')
            color = (cv.getTrackbarPos('B', 'controls'), cv.getTrackbarPos('G', 'controls'), cv.getTrackbarPos('R', 'controls'))
            if mode == 3:
                cv.polylines(img, [np.array(points, np.int32)], False, color, thickness)
            elif mode == 4:
                # Rellenar el polígono si el trackbar de relleno está activo
                if cv.getTrackbarPos('Relleno', 'controls') == 1:
                    cv.fillPoly(img, [np.array(points, np.int32)], color)
                else:
                    cv.polylines(img, [np.array(points, np.int32)], True, color, thickness)
            points.clear()

    elif event == cv.EVENT_LBUTTONUP:  # Al soltar el clic izquierdo se dibuja la figura
        drawing = False
        thickness = cv.getTrackbarPos('Grosor', 'controls')
        color = (cv.getTrackbarPos('B', 'controls'), cv.getTrackbarPos('G', 'controls'), cv.getTrackbarPos('R', 'controls'))

        # Si el trackbar de relleno esta activo, el grosor es -1 para rellenar la figura
        if cv.getTrackbarPos('Relleno', 'controls') == 1:
            thickness = -1  

        if mode == 0:  # Rectángulo
            cv.rectangle(img, (ix, iy), (x, y), color, thickness)
        elif mode == 1:  # Círculo
            radius = int(np.sqrt((x - ix) ** 2 + (y - iy) ** 2))
            cv.circle(img, (ix, iy), radius, color, thickness)
        elif mode == 2:  # Línea
            # En la línea el grosor se toma del trackbar porque no puede tener relleno
            cv.line(img, (ix, iy), (x, y), color, cv.getTrackbarPos('Grosor', 'controls')) 
        elif mode == 5:  # Elipse
            cv.ellipse(img, (ix, iy), (abs(x - ix), abs(y - iy)), 0, 0, 360, color, thickness)

        if video_writer:
            video_writer.write(img)

# Configuración inicial
img = np.zeros((800, 950, 3), np.uint8)
cv.namedWindow('canvas')  # Ventana del lienzo
cv.namedWindow('controls')  # Ventana de los controles
cv.namedWindow('legend')    # Ventana para la leyenda de teclas
cv.setMouseCallback('canvas', draw_figures)  

# Trackbars en la ventana de controles
cv.createTrackbar('Relleno', 'controls', 0, 1, trackbar)
cv.createTrackbar('R', 'controls', 255, 255, trackbar)
cv.createTrackbar('G', 'controls', 0, 255, trackbar)
cv.createTrackbar('B', 'controls', 0, 255, trackbar)
cv.createTrackbar('Grosor', 'controls', 1, 10, trackbar)

initialize_video_writer()  # Inicializar el video writer

# Creamos una imagen para la leyenda de las teclas
legend_img = np.zeros((250, 250, 3), np.uint8)
cv.putText(legend_img, 'Teclas de acceso rapido:', (10, 20), cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
cv.putText(legend_img, 'c - Circulo', (10, 50), cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
cv.putText(legend_img, 'l - Linea', (10, 70), cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
cv.putText(legend_img, 'r - Rectangulo', (10, 90), cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
cv.putText(legend_img, 'p - Polilinea', (10, 110), cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
cv.putText(legend_img, 'o - Poligono', (10, 130), cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
cv.putText(legend_img, 'e - Elipse', (10, 150), cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
cv.putText(legend_img, 's - Guardar Imagen', (10, 170), cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
cv.putText(legend_img, 'n - Limpiar Lienzo', (10, 190), cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
cv.putText(legend_img, 'ESC - Salir', (10, 210), cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

# Mostrar la ventana de leyenda
cv.imshow('legend', legend_img)


while True:
    img_copy = img.copy()
    cv.putText(img_copy, f'Modo: {mode_names[mode]}', (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv.LINE_AA)
    cv.imshow('canvas', img_copy)

    k = cv.waitKey(1) & 0xFF
    if k == ord('c'):  # Cambiar a modo círculo
        mode = 1
    elif k == ord('l'):  # Cambiar a modo línea
        mode = 2
    elif k == ord('r'):  # Cambiar a modo rectángulo
        mode = 0
    elif k == ord('p'):  # Cambiar a modo polilínea
        mode = 3
    elif k == ord('o'):  # Cambiar a modo polígono
        mode = 4
    elif k == ord('e'):  # Cambiar a modo elipse
        mode = 5
    elif k == ord('s'):  # Guardar la imagen actual
        cv.imwrite('drawing.png', img)
    elif k == ord('n'):  # Limpiar el lienzo
        img = np.zeros((800, 950, 3), np.uint8)
        points = []
    elif k == 27:  # ESC
        break

video_writer.release()  # Liberar el video writer
cv.destroyAllWindows()


El funcionamiento principal del código gira en torno a dos ventanas: canvas, donde se muestra el lienzo de dibujo, y controls, que contiene varios trackbars para ajustar el color, el grosor de las líneas y la opción de rellenar las figuras. Además, se incluye una leyenda con las teclas de acceso rápido para cada figura.

El proceso de dibujo se activa a través de un callback del ratón (draw_figures), que detecta eventos de clic y movimiento. Al hacer clic izquierdo, se inicia la creación de la figura seleccionada y se permite al usuario ajustar sus dimensiones arrastrando el ratón sin soltar el clic izquierdo. En el caso de las polilíneas y polígonos, haciendo clic izquierdo se seleccionan los puntos y con el clic derecho se forma la figura. Las figuras se dibujan en tiempo real con los valores especificados en los trackbars, y el usuario puede cambiar entre los modos de figura usando teclas específicas (c, l, r, p, o, e) como se indica en la leyenda. La tecla s permite guardar una captura de la imagen, mientras que n limpia el lienzo. Cada figura terminada se agrega a un archivo de video mediante el VideoWriter, creando una grabación del proceso de dibujo.

Finalmente, al presionar ESC, el programa se cierra y libera todos los recursos utilizados. En un primer instante, habíamos realizado el ejercicio en una única ventana para los trackbars y el lienzo, pero el espacio de dibujo se quedaba corto y no permitía una visualización adecuada de las figuras. Por ello, decidimos separar las ventanas y permitir una mayor flexibilidad en el dibujo.